In [1]:
# 인구증감률   
# 천명당 기초생활수급자수
# 노령화지수

# 인구사회 데이터 불러오기

In [2]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 인구증감률
GetCompasData('SBJ_2208_001', '5', '5_연수구_성연령별_주민등록인구.xlsx') # 인구증감률
pd1=pd.read_excel('5_연수구_성연령별_주민등록인구.xlsx') # 인구증감률

#노령화지수
## 65세 이상
pd2_1 = pd.read_csv('행정구역_읍면동_별_5세별_주민등록인구_2011-2021(65세_이상).csv',encoding='cp949')
## 15세 미만
pd2_2 = pd.read_csv('행정구역_읍면동_별_5세별_주민등록인구_2011_2021(15세_미만).csv', encoding='cp949')


# 천명당 기초생활수급자수
GetCompasData('SBJ_2208_001', '7', '7_연수구_기초생활수급대상자.xlsx')    # 천명당 기초생활수급자수               
pd3 = pd.read_excel('7_연수구_기초생활수급대상자.xlsx', sheet_name=None)
pd3 = pd.concat([value.assign(sheet_source=key) for key,value in pd3.items()], ignore_index=True)



[2022-09-30 14:11:40] 성공 - 5_연수구_성연령별_주민등록인구.xlsx
[2022-09-30 14:11:41] 성공 - 7_연수구_기초생활수급대상자.xlsx


# 데이터 확인 & 전처리


## 1. 인구증감률

In [4]:
# m_total_pop 남성 주민등록인구 총 합
# w_total_pop 여성 주민등록인구 총 합

total_pop = pd1.copy()
total_pop.head()


,emd_nm,base_date,m_total_pop,m_0g_pop,m_10g_pop,m_20g_pop,m_30g_pop,m_40g_pop,m_50g_pop,m_60g_pop,...,w_10g_pop,w_20g_pop,w_30g_pop,w_40g_pop,w_50g_pop,w_60g_pop,w_70g_pop,w_80g_pop,w_90g_pop,w_100g_pop
0,인천광역시 연수구,201301,146929,14381,23266,21980,21531,28205,24229,8053,...,21092,20515,22217,30103,21469,8000,6292,2921,408,15
1,인천광역시 연수구 옥련1동,201301,11392,1074,1842,1593,1619,2204,1812,733,...,1655,1453,1693,2371,1623,768,569,244,36,2
2,인천광역시 연수구 옥련2동,201301,12072,1159,1987,1626,1718,2387,1961,746,...,1843,1590,1807,2564,1767,743,564,256,36,3
3,인천광역시 연수구 선학동,201301,10565,688,1252,1931,1766,1651,2024,794,...,1136,1719,1409,1803,1957,759,570,258,36,1
4,인천광역시 연수구 연수1동,201301,14335,1075,1879,2440,2644,2787,2244,792,...,1734,2072,1981,2505,1837,762,505,219,34,1


In [5]:
total_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   emd_nm       1639 non-null   object
 1   base_date    1639 non-null   int64 
 2   m_total_pop  1639 non-null   int64 
 3   m_0g_pop     1639 non-null   int64 
 4   m_10g_pop    1639 non-null   int64 
 5   m_20g_pop    1639 non-null   int64 
 6   m_30g_pop    1639 non-null   int64 
 7   m_40g_pop    1639 non-null   int64 
 8   m_50g_pop    1639 non-null   int64 
 9   m_60g_pop    1639 non-null   int64 
 10  m_70g_pop    1639 non-null   int64 
 11  m_80g_pop    1639 non-null   int64 
 12  m_90g_pop    1639 non-null   int64 
 13  m_100g_pop   1639 non-null   int64 
 14  w_total_pop  1639 non-null   int64 
 15  w_0g_pop     1639 non-null   int64 
 16  w_10g_pop    1639 non-null   int64 
 17  w_20g_pop    1639 non-null   int64 
 18  w_30g_pop    1639 non-null   int64 
 19  w_40g_pop    1639 non-null 

In [6]:
# 컬럼 확인
total_pop.columns

Index(['emd_nm', 'base_date', 'm_total_pop', 'm_0g_pop', 'm_10g_pop',
       'm_20g_pop', 'm_30g_pop', 'm_40g_pop', 'm_50g_pop', 'm_60g_pop',
       'm_70g_pop', 'm_80g_pop', 'm_90g_pop', 'm_100g_pop', 'w_total_pop',
       'w_0g_pop', 'w_10g_pop', 'w_20g_pop', 'w_30g_pop', 'w_40g_pop',
       'w_50g_pop', 'w_60g_pop', 'w_70g_pop', 'w_80g_pop', 'w_90g_pop',
       'w_100g_pop'],
      dtype='object')

In [7]:
# 총 인구수(남+여) 컬럼 'total_pop' 생성 및 기타 불필요한 컬럼 삭제

total_pop['total_pop'] = total_pop['m_total_pop'] + total_pop['w_total_pop']
total_pop= total_pop[['emd_nm', 'base_date','total_pop']]
total_pop

,emd_nm,base_date,total_pop
0,인천광역시 연수구,201301,293489
1,인천광역시 연수구 옥련1동,201301,22860
2,인천광역시 연수구 옥련2동,201301,24348
3,인천광역시 연수구 선학동,201301,20861
4,인천광역시 연수구 연수1동,201301,26931
...,...,...,...
1634,인천광역시 연수구 송도1동,202206,38036
1635,인천광역시 연수구 송도2동,202206,36496
1636,인천광역시 연수구 송도3동,202206,48773
1637,인천광역시 연수구 송도4동,202206,25553


In [8]:
# base_date 컬럼 구성 확인 
# 202206이 마지막 지점이므로, 모든 연도의 6월을 기준으로 해당 연도의 인구수를 산출하는 것이 단위 통일성을 위해 좋을 듯 

total_pop['base_date'].unique()

array([201301, 201302, 201303, 201304, 201305, 201306, 201307, 201308,
       201309, 201310, 201311, 201312, 201401, 201402, 201403, 201404,
       201405, 201406, 201407, 201408, 201409, 201410, 201411, 201412,
       201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508,
       201509, 201510, 201511, 201512, 201601, 201602, 201603, 201604,
       201605, 201606, 201607, 201608, 201609, 201610, 201611, 201612,
       201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106, 202107, 202108,
       202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204,
      

In [9]:
# ['base_date'] 값이 특정 값인 경우만 추출하여 데이터프레임 재구조화 

total_pop = total_pop[
                (total_pop['base_date'] == 201306)|
                (total_pop['base_date'] == 201406)|
                (total_pop['base_date'] == 201506)|
                (total_pop['base_date'] == 201606)|
                (total_pop['base_date'] == 201706)|
                (total_pop['base_date'] == 201806)|
                (total_pop['base_date'] == 201906)|
                (total_pop['base_date'] == 202006)|
                (total_pop['base_date'] == 202106)|
                (total_pop['base_date'] == 202206)]

total_pop

,emd_nm,base_date,total_pop
65,인천광역시 연수구,201306,295586
66,인천광역시 연수구 옥련1동,201306,22786
67,인천광역시 연수구 옥련2동,201306,24160
68,인천광역시 연수구 선학동,201306,20812
69,인천광역시 연수구 연수1동,201306,26813
...,...,...,...
1634,인천광역시 연수구 송도1동,202206,38036
1635,인천광역시 연수구 송도2동,202206,36496
1636,인천광역시 연수구 송도3동,202206,48773
1637,인천광역시 연수구 송도4동,202206,25553


In [10]:
# 결측치 없음

total_pop.isnull().sum(), total_pop.isna().sum()

(emd_nm       0
 base_date    0
 total_pop    0
 dtype: int64,
 emd_nm       0
 base_date    0
 total_pop    0
 dtype: int64)

In [11]:
# 인덱스 재설정 

total_pop.reset_index(inplace=True, drop=True)
total_pop

,emd_nm,base_date,total_pop
0,인천광역시 연수구,201306,295586
1,인천광역시 연수구 옥련1동,201306,22786
2,인천광역시 연수구 옥련2동,201306,24160
3,인천광역시 연수구 선학동,201306,20812
4,인천광역시 연수구 연수1동,201306,26813
...,...,...,...
139,인천광역시 연수구 송도1동,202206,38036
140,인천광역시 연수구 송도2동,202206,36496
141,인천광역시 연수구 송도3동,202206,48773
142,인천광역시 연수구 송도4동,202206,25553


In [12]:
# 동 이름 split, 연도만 추출

total_pop['emd_nm'] = total_pop['emd_nm'].str.split().str[-1]
total_pop['base_date'] = total_pop['base_date']//100
total_pop

,emd_nm,base_date,total_pop
0,연수구,2013,295586
1,옥련1동,2013,22786
2,옥련2동,2013,24160
3,선학동,2013,20812
4,연수1동,2013,26813
...,...,...,...
139,송도1동,2022,38036
140,송도2동,2022,36496
141,송도3동,2022,48773
142,송도4동,2022,25553


In [13]:
# 데이터프레임 재구조화

total_pop = total_pop.pivot(index='emd_nm', columns='base_date',values='total_pop').reset_index().rename_axis(None, axis=1)
total_pop


,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,17096.0,16788.0,16716.0,15976.0,15618.0,17976.0,23149.0,26213.0,25582.0,25267.0
1,동춘2동,21851.0,21538.0,21379.0,20975.0,20751.0,20145.0,19386.0,19112.0,18459.0,18118.0
2,동춘3동,20628.0,20531.0,20415.0,20145.0,19974.0,19685.0,19337.0,19103.0,18739.0,18337.0
3,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
4,송도1동,38763.0,51217.0,35012.0,34549.0,35053.0,34629.0,34052.0,36582.0,38082.0,38036.0
5,송도2동,26358.0,27214.0,31843.0,45852.0,46896.0,53878.0,36595.0,36673.0,36798.0,36496.0
6,송도3동,NaN,NaN,19442.0,23249.0,34175.0,38442.0,47696.0,48828.0,49000.0,48773.0
7,송도4동,NaN,NaN,NaN,NaN,NaN,NaN,26052.0,41679.0,25172.0,25553.0
8,송도5동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39114.0,40056.0
9,연수1동,26813.0,26610.0,25944.0,25286.0,24079.0,22818.0,21589.0,20678.0,19777.0,18958.0


In [14]:
# None -> 0
total_pop= total_pop.replace({None:0})
total_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   emd_nm  16 non-null     object 
 1   2013    16 non-null     float64
 2   2014    16 non-null     float64
 3   2015    16 non-null     float64
 4   2016    16 non-null     float64
 5   2017    16 non-null     float64
 6   2018    16 non-null     float64
 7   2019    16 non-null     float64
 8   2020    16 non-null     float64
 9   2021    16 non-null     float64
 10  2022    16 non-null     float64
dtypes: float64(10), object(1)
memory usage: 1.5+ KB


In [15]:
total_pop

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,17096.0,16788.0,16716.0,15976.0,15618.0,17976.0,23149.0,26213.0,25582.0,25267.0
1,동춘2동,21851.0,21538.0,21379.0,20975.0,20751.0,20145.0,19386.0,19112.0,18459.0,18118.0
2,동춘3동,20628.0,20531.0,20415.0,20145.0,19974.0,19685.0,19337.0,19103.0,18739.0,18337.0
3,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
4,송도1동,38763.0,51217.0,35012.0,34549.0,35053.0,34629.0,34052.0,36582.0,38082.0,38036.0
5,송도2동,26358.0,27214.0,31843.0,45852.0,46896.0,53878.0,36595.0,36673.0,36798.0,36496.0
6,송도3동,0.0,0.0,19442.0,23249.0,34175.0,38442.0,47696.0,48828.0,49000.0,48773.0
7,송도4동,0.0,0.0,0.0,0.0,0.0,0.0,26052.0,41679.0,25172.0,25553.0
8,송도5동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39114.0,40056.0
9,연수1동,26813.0,26610.0,25944.0,25286.0,24079.0,22818.0,21589.0,20678.0,19777.0,18958.0


In [16]:
# 연수구제거 ( 연수 추출 위함 )
yeonsu_gu = total_pop[total_pop['emd_nm'] == '연수구']
total_pop.drop(12,inplace=True)
total_pop.reset_index(inplace=True, drop=True)

In [17]:
yeonsu_gu

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
12,연수구,295586.0,306754.0,311316.0,324103.0,332849.0,339618.0,355218.0,372566.0,390827.0,387303.0


In [18]:
total_pop

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,17096.0,16788.0,16716.0,15976.0,15618.0,17976.0,23149.0,26213.0,25582.0,25267.0
1,동춘2동,21851.0,21538.0,21379.0,20975.0,20751.0,20145.0,19386.0,19112.0,18459.0,18118.0
2,동춘3동,20628.0,20531.0,20415.0,20145.0,19974.0,19685.0,19337.0,19103.0,18739.0,18337.0
3,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
4,송도1동,38763.0,51217.0,35012.0,34549.0,35053.0,34629.0,34052.0,36582.0,38082.0,38036.0
5,송도2동,26358.0,27214.0,31843.0,45852.0,46896.0,53878.0,36595.0,36673.0,36798.0,36496.0
6,송도3동,0.0,0.0,19442.0,23249.0,34175.0,38442.0,47696.0,48828.0,49000.0,48773.0
7,송도4동,0.0,0.0,0.0,0.0,0.0,0.0,26052.0,41679.0,25172.0,25553.0
8,송도5동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39114.0,40056.0
9,연수1동,26813.0,26610.0,25944.0,25286.0,24079.0,22818.0,21589.0,20678.0,19777.0,18958.0


In [19]:
# OOn동 -> OO으로  ->  OO별 합 구하기 위함
total_pop['emd_nm'] = total_pop['emd_nm'].str[:2]
total_pop

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘,17096.0,16788.0,16716.0,15976.0,15618.0,17976.0,23149.0,26213.0,25582.0,25267.0
1,동춘,21851.0,21538.0,21379.0,20975.0,20751.0,20145.0,19386.0,19112.0,18459.0,18118.0
2,동춘,20628.0,20531.0,20415.0,20145.0,19974.0,19685.0,19337.0,19103.0,18739.0,18337.0
3,선학,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
4,송도,38763.0,51217.0,35012.0,34549.0,35053.0,34629.0,34052.0,36582.0,38082.0,38036.0
5,송도,26358.0,27214.0,31843.0,45852.0,46896.0,53878.0,36595.0,36673.0,36798.0,36496.0
6,송도,0.0,0.0,19442.0,23249.0,34175.0,38442.0,47696.0,48828.0,49000.0,48773.0
7,송도,0.0,0.0,0.0,0.0,0.0,0.0,26052.0,41679.0,25172.0,25553.0
8,송도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39114.0,40056.0
9,연수,26813.0,26610.0,25944.0,25286.0,24079.0,22818.0,21589.0,20678.0,19777.0,18958.0


In [20]:
# 같은 동끼리 합 구하기
total_pop2 = total_pop.groupby('emd_nm').sum()
total_pop2

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
emd_nm,,,,,,,,,,
동춘,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0
선학,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
송도,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0
연수,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0
옥련,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0
청학,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0


In [21]:
total_pop2.reset_index(inplace=True)

# 동 붙이기
total_pop2['emd_nm'] = total_pop2['emd_nm']+'동'




In [22]:
total_pop2.reset_index(drop=True, inplace=True)
total_pop2

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0


In [23]:
# 인구증감률 2013-2022
# 인구만 시각화 하기 위해 drop 진행
tableau_총인구수 = total_pop2.copy()
tableau_총인구수

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0


In [24]:
# 위의 데이터프레임을 dong, year, 인구수로 재구성 (melt)

tableau_총인구수 = pd.melt(tableau_총인구수,
               id_vars = 'emd_nm',
               var_name = 'year',
               value_name = '총인구수')

# dong과 year을 기준으로 정렬
tableau_총인구수= tableau_총인구수.sort_values(by=['emd_nm','year'])

# 인덱스 초기화
tableau_총인구수.reset_index(drop=True, inplace=True)
tableau_총인구수

,emd_nm,year,총인구수
0,동춘동,2013,59575.0
1,동춘동,2014,58857.0
2,동춘동,2015,58510.0
3,동춘동,2016,57096.0
4,동춘동,2017,56343.0
5,동춘동,2018,57806.0
6,동춘동,2019,61872.0
7,동춘동,2020,64428.0
8,동춘동,2021,62780.0
9,동춘동,2022,61722.0


In [25]:
total_pop2

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0


In [26]:
# 인구증감률 구하기
total_pop2['9년 인구증감률'] =  (total_pop2[2022] - total_pop2[2013]) / total_pop2[2013]*100
total_pop2

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감률
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870


In [27]:
# 인구상대감소지수 = (최대 - 원래) / (최대 - 최소) *100

max1 = total_pop2['9년 인구증감률'].max()
min1 = total_pop2['9년 인구증감률'].min()

total_pop2['인구상대감소지수'] = (max1 - total_pop2['9년 인구증감률']) / (max1-min1) * 100
total_pop2


,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감률,인구상대감소지수
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861,87.580280
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697,97.847306
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897,0.000000
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633,100.000000
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831,96.952280
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870,98.764549


In [28]:
# 인구증감률 표준화 (쇠퇴진단지표 - 하한값) / (상한값 - 하한값) 

max2 = total_pop2['인구상대감소지수'].max()
min2 = total_pop2['인구상대감소지수'].min()
total_pop2['std'] = (total_pop2['인구상대감소지수'] - min2) / (max2 - min2)
total_pop2

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감률,인구상대감소지수,std
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861,87.580280,0.875803
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697,97.847306,0.978473
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897,0.000000,0.000000
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633,100.000000,1.000000
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831,96.952280,0.969523
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870,98.764549,0.987645


In [29]:
total_pop_final = total_pop2.drop(['인구상대감소지수'],axis=1).copy()
total_pop_final.columns=['dong','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','9년 인구증감율','인구상대감소지수_std']
total_pop_final

,dong,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감율,인구상대감소지수_std
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861,0.875803
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697,0.978473
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897,0.000000
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633,1.000000
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831,0.969523
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870,0.987645


## 천명당 기초생활수급자수

In [30]:
basic = pd3.copy()
basic

,emd_nm,total_pop,sheet_source
0,합 계,7995,2022_기초수급자현황
1,옥련1동,605,2022_기초수급자현황
2,옥련2동,331,2022_기초수급자현황
3,선학동,1249,2022_기초수급자현황
4,연수1동,1048,2022_기초수급자현황
...,...,...,...
149,청학동,1018,2012_기초수급자현황
150,동춘1동,153,2012_기초수급자현황
151,동춘2동,91,2012_기초수급자현황
152,동춘3동,21,2012_기초수급자현황


In [31]:
# sheet_source 2022_기초수급자현황 -> 2022 연도만 슬라이싱

basic['sheet_source'] = basic['sheet_source'].str.split('_').str[0]

In [32]:
# DataFrame 보기 좋게 재구성

basic =  basic.pivot_table(index = 'emd_nm', columns='sheet_source',values='total_pop').reset_index().rename_axis(None, axis=1)
basic

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,153.0,182.0,173.0,233.0,271.0,244.0,214.0,240.0,294.0,406.0,193.0
1,동춘2동,91.0,96.0,97.0,96.0,164.0,156.0,137.0,204.0,252.0,344.0,148.0
2,동춘3동,21.0,18.0,28.0,21.0,50.0,51.0,44.0,108.0,154.0,194.0,58.0
3,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
4,송도1동,NaN,NaN,59.0,42.0,97.0,92.0,79.0,105.0,149.0,228.0,57.0
5,송도2동,NaN,NaN,9.0,12.0,52.0,69.0,73.0,119.0,77.0,80.0,33.0
6,송도3동,NaN,NaN,NaN,12.0,35.0,55.0,58.0,85.0,122.0,175.0,80.0
7,송도4동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,80.0,43.0
8,송도5동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,52.0
9,송도동,56.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# None -> 0
basic = basic.replace({None:0})
basic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   emd_nm  17 non-null     object 
 1   2012    17 non-null     float64
 2   2013    17 non-null     float64
 3   2014    17 non-null     float64
 4   2015    17 non-null     float64
 5   2016    17 non-null     float64
 6   2017    17 non-null     float64
 7   2018    17 non-null     float64
 8   2019    17 non-null     float64
 9   2020    17 non-null     float64
 10  2021    17 non-null     float64
 11  2022    17 non-null     float64
dtypes: float64(11), object(1)
memory usage: 1.7+ KB


In [34]:
basic

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,153.0,182.0,173.0,233.0,271.0,244.0,214.0,240.0,294.0,406.0,193.0
1,동춘2동,91.0,96.0,97.0,96.0,164.0,156.0,137.0,204.0,252.0,344.0,148.0
2,동춘3동,21.0,18.0,28.0,21.0,50.0,51.0,44.0,108.0,154.0,194.0,58.0
3,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
4,송도1동,0.0,0.0,59.0,42.0,97.0,92.0,79.0,105.0,149.0,228.0,57.0
5,송도2동,0.0,0.0,9.0,12.0,52.0,69.0,73.0,119.0,77.0,80.0,33.0
6,송도3동,0.0,0.0,0.0,12.0,35.0,55.0,58.0,85.0,122.0,175.0,80.0
7,송도4동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,80.0,43.0
8,송도5동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.0,52.0
9,송도동,56.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# 합계 제거
basic.drop(16,inplace=True)
basic

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘1동,153.0,182.0,173.0,233.0,271.0,244.0,214.0,240.0,294.0,406.0,193.0
1,동춘2동,91.0,96.0,97.0,96.0,164.0,156.0,137.0,204.0,252.0,344.0,148.0
2,동춘3동,21.0,18.0,28.0,21.0,50.0,51.0,44.0,108.0,154.0,194.0,58.0
3,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
4,송도1동,0.0,0.0,59.0,42.0,97.0,92.0,79.0,105.0,149.0,228.0,57.0
5,송도2동,0.0,0.0,9.0,12.0,52.0,69.0,73.0,119.0,77.0,80.0,33.0
6,송도3동,0.0,0.0,0.0,12.0,35.0,55.0,58.0,85.0,122.0,175.0,80.0
7,송도4동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,80.0,43.0
8,송도5동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.0,52.0
9,송도동,56.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
basic['emd_nm'] = basic['emd_nm'].str[:2]
basic

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘,153.0,182.0,173.0,233.0,271.0,244.0,214.0,240.0,294.0,406.0,193.0
1,동춘,91.0,96.0,97.0,96.0,164.0,156.0,137.0,204.0,252.0,344.0,148.0
2,동춘,21.0,18.0,28.0,21.0,50.0,51.0,44.0,108.0,154.0,194.0,58.0
3,선학,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
4,송도,0.0,0.0,59.0,42.0,97.0,92.0,79.0,105.0,149.0,228.0,57.0
5,송도,0.0,0.0,9.0,12.0,52.0,69.0,73.0,119.0,77.0,80.0,33.0
6,송도,0.0,0.0,0.0,12.0,35.0,55.0,58.0,85.0,122.0,175.0,80.0
7,송도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,80.0,43.0
8,송도,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.0,52.0
9,송도,56.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
basic2 = basic.groupby('emd_nm').sum().copy()

basic2.reset_index(inplace=True)
basic2

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘,265.0,296.0,298.0,350.0,485.0,451.0,395.0,552.0,700.0,944.0,399.0
1,선학,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
2,송도,56.0,68.0,68.0,66.0,184.0,216.0,210.0,309.0,452.0,669.0,265.0
3,연수,4126.0,3966.0,3913.0,3897.0,4821.0,4744.0,4268.0,4501.0,4554.0,4945.0,3534.0
4,옥련,749.0,695.0,706.0,710.0,1032.0,1155.0,1177.0,1274.0,1439.0,1622.0,936.0
5,청학,1018.0,1183.0,1183.0,1300.0,1561.0,1523.0,1505.0,1627.0,1732.0,1990.0,1312.0


In [38]:
# 기초생활수급자수 동, 연도 별 확인
basic2['emd_nm'] = basic2['emd_nm'] + '동'
basic2

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘동,265.0,296.0,298.0,350.0,485.0,451.0,395.0,552.0,700.0,944.0,399.0
1,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
2,송도동,56.0,68.0,68.0,66.0,184.0,216.0,210.0,309.0,452.0,669.0,265.0
3,연수동,4126.0,3966.0,3913.0,3897.0,4821.0,4744.0,4268.0,4501.0,4554.0,4945.0,3534.0
4,옥련동,749.0,695.0,706.0,710.0,1032.0,1155.0,1177.0,1274.0,1439.0,1622.0,936.0
5,청학동,1018.0,1183.0,1183.0,1300.0,1561.0,1523.0,1505.0,1627.0,1732.0,1990.0,1312.0


In [39]:
total_pop2

,emd_nm,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감률,인구상대감소지수,std
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861,87.580280,0.875803
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697,97.847306,0.978473
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897,0.000000,0.000000
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633,100.000000,1.000000
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831,96.952280,0.969523
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870,98.764549,0.987645


In [40]:
# 2021년 천명당 기초생활 수급자
basic2['2021년 천명당 기초생활수급자'] = basic2['2021'] / total_pop2[2021] * 1000
basic2

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2021년 천명당 기초생활수급자
0,동춘동,265.0,296.0,298.0,350.0,485.0,451.0,395.0,552.0,700.0,944.0,399.0,15.036636
1,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0,94.010176
2,송도동,56.0,68.0,68.0,66.0,184.0,216.0,210.0,309.0,452.0,669.0,265.0,3.555371
3,연수동,4126.0,3966.0,3913.0,3897.0,4821.0,4744.0,4268.0,4501.0,4554.0,4945.0,3534.0,86.000000
4,옥련동,749.0,695.0,706.0,710.0,1032.0,1155.0,1177.0,1274.0,1439.0,1622.0,936.0,40.354282
5,청학동,1018.0,1183.0,1183.0,1300.0,1561.0,1523.0,1505.0,1627.0,1732.0,1990.0,1312.0,79.948576


In [41]:
# 기초생활수급자수 시각화를 위한 수만 추출하기.

tableau_기초생활수급자수 = basic2.drop(['2021년 천명당 기초생활수급자'],axis=1).copy()
tableau_기초생활수급자수

,emd_nm,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,동춘동,265.0,296.0,298.0,350.0,485.0,451.0,395.0,552.0,700.0,944.0,399.0
1,선학동,1639.0,1441.0,1444.0,1353.0,1453.0,1492.0,1452.0,1432.0,1460.0,1626.0,1249.0
2,송도동,56.0,68.0,68.0,66.0,184.0,216.0,210.0,309.0,452.0,669.0,265.0
3,연수동,4126.0,3966.0,3913.0,3897.0,4821.0,4744.0,4268.0,4501.0,4554.0,4945.0,3534.0
4,옥련동,749.0,695.0,706.0,710.0,1032.0,1155.0,1177.0,1274.0,1439.0,1622.0,936.0
5,청학동,1018.0,1183.0,1183.0,1300.0,1561.0,1523.0,1505.0,1627.0,1732.0,1990.0,1312.0


In [42]:
# 기초생활수급자수를 emd_nm, year, 기초생활수급자수로 재구성

tableau_기초생활수급자수 = pd.melt(tableau_기초생활수급자수,
               id_vars = 'emd_nm',
               var_name = 'year',
               value_name = '기초생활수급자수')

# 동 이름, 연도로 정렬
tableau_기초생활수급자수= tableau_기초생활수급자수.sort_values(by=['emd_nm','year'])

# 컬렴명 변경 후 인덱스 초기화
tableau_기초생활수급자수.columns=['dong','year','기초생활수급자수']
tableau_기초생활수급자수.reset_index(drop=True,inplace=True)
tableau_기초생활수급자수

,dong,year,기초생활수급자수
0,동춘동,2012,265.0
1,동춘동,2013,296.0
2,동춘동,2014,298.0
3,동춘동,2015,350.0
4,동춘동,2016,485.0
...,...,...,...
61,청학동,2018,1505.0
62,청학동,2019,1627.0
63,청학동,2020,1732.0
64,청학동,2021,1990.0


In [43]:
# 천명당기초생활수급자수를 위해 2021년도 데이터만 추출
basic_final = basic2[['emd_nm','2021년 천명당 기초생활수급자']]

# 천명당기초생활수급자수 표준화 (쇠퇴진단지표 - 하한값) / (상한값 - 하한값) 
max2 = basic_final['2021년 천명당 기초생활수급자'].max()
min2 = basic_final['2021년 천명당 기초생활수급자'].min()

# 표준화 작업
basic_final['2021년 천명당 기초생활수급자_std'] = (basic_final['2021년 천명당 기초생활수급자'] - min2) / (max2 - min2)
basic_final

,emd_nm,2021년 천명당 기초생활수급자,2021년 천명당 기초생활수급자_std
0,동춘동,15.036636,0.126928
1,선학동,94.010176,1.000000
2,송도동,3.555371,0.000000
3,연수동,86.000000,0.911446
4,옥련동,40.354282,0.406821
5,청학동,79.948576,0.844546


## 노령화지수

In [44]:
# 65세이상

# 2011.n -> 2011은 연도, n은 나이 구간을 나타냄
senior = pd2_1.copy()
senior.head()

,행정구역(동읍면)별,항목,2011,2011.1,2011.2,2011.3,2011.4,2011.5,2011.6,2011.7,...,2020.6,2020.7,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
0,행정구역(동읍면)별,항목,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+,...,95 - 99세,100+,65 - 69세,70 - 74세,75 - 79세,80 - 84세,85 - 89세,90 - 94세,95 - 99세,100+
1,옥련1동,총인구수 (명),653,557,362,225,103,34,5,1,...,15,3,1030,731,589,442,215,90,16,0
2,옥련2동,총인구수 (명),660,558,363,220,109,32,10,4,...,22,5,1057,665,531,368,185,81,18,4
3,선학동,총인구수 (명),609,513,373,233,92,32,7,1,...,16,4,1238,734,516,402,205,81,19,5
4,연수1동,총인구수 (명),657,457,366,198,85,29,4,0,...,15,4,1089,660,505,331,185,67,14,3


In [45]:
# 필요없는 부분 지우기
senior.drop(['항목'], axis=1, inplace=True)
senior.drop(0,inplace=True)
senior.head()

,행정구역(동읍면)별,2011,2011.1,2011.2,2011.3,2011.4,2011.5,2011.6,2011.7,2012,...,2020.6,2020.7,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
1,옥련1동,653,557,362,225,103,34,5,1,678,...,15,3,1030,731,589,442,215,90,16,0
2,옥련2동,660,558,363,220,109,32,10,4,664,...,22,5,1057,665,531,368,185,81,18,4
3,선학동,609,513,373,233,92,32,7,1,614,...,16,4,1238,734,516,402,205,81,19,5
4,연수1동,657,457,366,198,85,29,4,0,691,...,15,4,1089,660,505,331,185,67,14,3
5,연수2동,545,506,413,275,124,45,3,1,558,...,32,5,1162,664,514,415,267,101,24,7


In [46]:
# 2011.n -> 2011 로 바꾸기
senior.columns = senior.columns.str[:4]
senior

,행정구역,2011,2011,2011,2011,2011,2011,2011,2011,2012,...,2020,2020,2021,2021,2021,2021,2021,2021,2021,2021
1,옥련1동,653,557,362,225,103,34,5,1,678,...,15,3,1030,731,589,442,215,90,16,0
2,옥련2동,660,558,363,220,109,32,10,4,664,...,22,5,1057,665,531,368,185,81,18,4
3,선학동,609,513,373,233,92,32,7,1,614,...,16,4,1238,734,516,402,205,81,19,5
4,연수1동,657,457,366,198,85,29,4,0,691,...,15,4,1089,660,505,331,185,67,14,3
5,연수2동,545,506,413,275,124,45,3,1,558,...,32,5,1162,664,514,415,267,101,24,7
6,연수3동,531,478,402,287,129,46,8,1,540,...,30,7,1048,645,532,440,278,109,32,5
7,청학동,870,735,490,263,137,48,7,3,886,...,35,9,1617,1005,765,582,321,111,30,8
8,동춘1동,423,407,309,220,118,62,19,6,403,...,24,14,1171,575,369,275,177,72,29,4
9,동춘2동,345,353,247,146,71,16,3,3,367,...,10,4,832,465,356,305,152,54,11,2
10,동춘3동,437,376,278,198,82,15,8,1,450,...,16,1,785,461,387,284,142,67,21,0


In [47]:
# 계산을 위해 '-'를 0으로 바꾸기

senior = senior.replace({'-':0})
senior

,행정구역,2011,2011,2011,2011,2011,2011,2011,2011,2012,...,2020,2020,2021,2021,2021,2021,2021,2021,2021,2021
1,옥련1동,653,557,362,225,103,34,5,1,678,...,15,3,1030,731,589,442,215,90,16,0
2,옥련2동,660,558,363,220,109,32,10,4,664,...,22,5,1057,665,531,368,185,81,18,4
3,선학동,609,513,373,233,92,32,7,1,614,...,16,4,1238,734,516,402,205,81,19,5
4,연수1동,657,457,366,198,85,29,4,0,691,...,15,4,1089,660,505,331,185,67,14,3
5,연수2동,545,506,413,275,124,45,3,1,558,...,32,5,1162,664,514,415,267,101,24,7
6,연수3동,531,478,402,287,129,46,8,1,540,...,30,7,1048,645,532,440,278,109,32,5
7,청학동,870,735,490,263,137,48,7,3,886,...,35,9,1617,1005,765,582,321,111,30,8
8,동춘1동,423,407,309,220,118,62,19,6,403,...,24,14,1171,575,369,275,177,72,29,4
9,동춘2동,345,353,247,146,71,16,3,3,367,...,10,4,832,465,356,305,152,54,11,2
10,동춘3동,437,376,278,198,82,15,8,1,450,...,16,1,785,461,387,284,142,67,21,0


In [48]:
# 동 만 추출

senior_dong = senior['행정구역']
senior.drop(['행정구역'],axis=1, inplace=True)
senior

,2011,2011,2011,2011,2011,2011,2011,2011,2012,2012,...,2020,2020,2021,2021,2021,2021,2021,2021,2021,2021
1,653,557,362,225,103,34,5,1,678,580,...,15,3,1030,731,589,442,215,90,16,0
2,660,558,363,220,109,32,10,4,664,568,...,22,5,1057,665,531,368,185,81,18,4
3,609,513,373,233,92,32,7,1,614,560,...,16,4,1238,734,516,402,205,81,19,5
4,657,457,366,198,85,29,4,0,691,525,...,15,4,1089,660,505,331,185,67,14,3
5,545,506,413,275,124,45,3,1,558,526,...,32,5,1162,664,514,415,267,101,24,7
6,531,478,402,287,129,46,8,1,540,524,...,30,7,1048,645,532,440,278,109,32,5
7,870,735,490,263,137,48,7,3,886,820,...,35,9,1617,1005,765,582,321,111,30,8
8,423,407,309,220,118,62,19,6,403,389,...,24,14,1171,575,369,275,177,72,29,4
9,345,353,247,146,71,16,3,3,367,343,...,10,4,832,465,356,305,152,54,11,2
10,437,376,278,198,82,15,8,1,450,392,...,16,1,785,461,387,284,142,67,21,0


In [49]:
# object -> int

senior = senior.apply(pd.to_numeric)


In [50]:
# 새로운 데이터 프레임 생성
senior2 = pd.DataFrame()

# 동 추가
senior2['dong'] = senior_dong

# 2011 끼리 합 ~ 2022끼리 합 
year = list(map(str, list(range(2011,2022))))
for y in year:
    senior2[y] = senior[y].sum(axis=1)
senior2

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,옥련1동,1940,2036,2114,2207,2286,2362,2502,2627,2774,2971,3113
2,옥련2동,1956,1999,2115,2130,2180,2270,2392,2466,2567,2726,2909
3,선학동,1860,1941,2027,2139,2240,2301,2467,2633,2801,3007,3200
4,연수1동,1796,1925,1971,2111,2140,2196,2338,2413,2546,2745,2854
5,연수2동,1912,1970,2100,2207,2253,2334,2433,2582,2763,2957,3154
6,연수3동,1882,1992,2098,2162,2232,2334,2484,2613,2742,3000,3089
7,청학동,2553,2782,2962,3070,3138,3284,3504,3708,3905,4194,4439
8,동춘1동,1564,1427,1469,1518,1504,1442,1497,1744,2288,2480,2672
9,동춘2동,1184,1238,1289,1345,1417,1485,1569,1686,1826,2024,2177
10,동춘3동,1395,1435,1505,1571,1606,1670,1745,1811,1921,1980,2147


In [51]:
# OOn동 -> OO 으로. ( 같은 OO끼리 묶기 위해)
senior2['dong'] = senior2['dong'].str[:2]
senior2

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,옥련,1940,2036,2114,2207,2286,2362,2502,2627,2774,2971,3113
2,옥련,1956,1999,2115,2130,2180,2270,2392,2466,2567,2726,2909
3,선학,1860,1941,2027,2139,2240,2301,2467,2633,2801,3007,3200
4,연수,1796,1925,1971,2111,2140,2196,2338,2413,2546,2745,2854
5,연수,1912,1970,2100,2207,2253,2334,2433,2582,2763,2957,3154
6,연수,1882,1992,2098,2162,2232,2334,2484,2613,2742,3000,3089
7,청학,2553,2782,2962,3070,3138,3284,3504,3708,3905,4194,4439
8,동춘,1564,1427,1469,1518,1504,1442,1497,1744,2288,2480,2672
9,동춘,1184,1238,1289,1345,1417,1485,1569,1686,1826,2024,2177
10,동춘,1395,1435,1505,1571,1606,1670,1745,1811,1921,1980,2147


In [52]:
senior3 = senior2.groupby('dong').sum()

senior3.reset_index(inplace=True)
senior3

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘,4143,4100,4263,4434,4527,4597,4811,5241,6035,6484,6996
1,선학,1860,1941,2027,2139,2240,2301,2467,2633,2801,3007,3200
2,송도,2493,2895,3344,3897,4597,5215,5991,6944,8213,10106,11267
3,연수,5590,5887,6169,6480,6625,6864,7255,7608,8051,8702,9097
4,옥련,3896,4035,4229,4337,4466,4632,4894,5093,5341,5697,6022
5,청학,2553,2782,2962,3070,3138,3284,3504,3708,3905,4194,4439


# 15세 미만


In [53]:
child = pd2_2.copy()

In [54]:
child.head()

,행정구역(동읍면)별,항목,2011,2011.1,2011.2,2012,2012.1,2012.2,2013,2013.1,...,2018.2,2019,2019.1,2019.2,2020,2020.1,2020.2,2021,2021.1,2021.2
0,행정구역(동읍면)별,항목,0 - 4세,5 - 9세,10 - 14세,0 - 4세,5 - 9세,10 - 14세,0 - 4세,5 - 9세,...,10 - 14세,0 - 4세,5 - 9세,10 - 14세,0 - 4세,5 - 9세,10 - 14세,0 - 4세,5 - 9세,10 - 14세
1,옥련1동,총인구수 (명),1016,1173,1715,1035,1121,1559,1039,1087,...,1047,604,926,945,519,885,911,467,783,885
2,옥련2동,총인구수 (명),1076,1249,1820,1035,1214,1654,1029,1190,...,1145,711,967,1094,611,937,1085,539,823,1058
3,선학동,총인구수 (명),687,640,1021,708,636,908,725,604,...,551,469,542,490,385,504,493,334,496,502
4,연수1동,총인구수 (명),1011,1044,1764,995,1015,1588,1039,972,...,807,451,692,713,385,575,676,337,527,653


In [55]:
# 필요없는 부분 drop
child.drop(['항목'],axis=1, inplace=True)
child.drop(0,inplace=True)
child

,행정구역(동읍면)별,2011,2011.1,2011.2,2012,2012.1,2012.2,2013,2013.1,2013.2,...,2018.2,2019,2019.1,2019.2,2020,2020.1,2020.2,2021,2021.1,2021.2
1,옥련1동,1016,1173,1715,1035,1121,1559,1039,1087,1415,...,1047,604,926,945,519,885,911,467,783,885
2,옥련2동,1076,1249,1820,1035,1214,1654,1029,1190,1475,...,1145,711,967,1094,611,937,1085,539,823,1058
3,선학동,687,640,1021,708,636,908,725,604,825,...,551,469,542,490,385,504,493,334,496,502
4,연수1동,1011,1044,1764,995,1015,1588,1039,972,1402,...,807,451,692,713,385,575,676,337,527,653
5,연수2동,1113,992,1453,1159,973,1300,1172,920,1196,...,875,784,791,822,661,736,772,597,708,737
6,연수3동,1039,893,1213,1052,867,1085,1054,830,1024,...,647,591,584,602,481,540,530,408,481,539
7,청학동,1293,1185,1743,1243,1160,1523,1195,1092,1410,...,916,659,784,830,543,729,804,450,699,772
8,동춘1동,602,845,1355,586,841,1249,595,775,1182,...,965,1162,1212,1239,1133,1232,1255,1019,1251,1294
9,동춘2동,1181,1229,1750,1162,1210,1585,1168,1161,1467,...,1065,708,890,972,603,785,926,546,730,913
10,동춘3동,659,1195,1888,668,1164,1837,701,1233,1770,...,1394,553,1155,1375,488,1076,1408,432,1009,1382


In [56]:
child.columns = child.columns.str[:4]

child = child.replace({'-':0})

child_dong = child['행정구역']

child.drop(['행정구역'],axis=1, inplace=True)

child = child.apply(pd.to_numeric)

# child2 생성
child2 = pd.DataFrame()
child2['dong'] = child_dong

year = list(map(str, list(range(2011,2022))))
for y in year:
    child2[y] = child[y].sum(axis=1)
child2

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,옥련1동,3904,3715,3541,3332,3194,3067,2949,2765,2475,2315,2135
2,옥련2동,4145,3903,3694,3523,3387,3285,3172,2956,2772,2633,2420
3,선학동,2348,2252,2154,1987,1868,1790,1722,1609,1501,1382,1332
4,연수1동,3819,3598,3413,3229,2987,2717,2486,2162,1856,1636,1517
5,연수2동,3558,3432,3288,3119,2980,2848,2768,2586,2397,2169,2042
6,연수3동,3145,3004,2908,2798,2521,2332,2191,2014,1777,1551,1428
7,청학동,4221,3926,3697,3547,3287,3002,2842,2564,2273,2076,1921
8,동춘1동,2802,2676,2552,2366,2268,2162,2005,2404,3613,3620,3564
9,동춘2동,4160,3957,3796,3666,3469,3288,3183,2858,2570,2314,2189
10,동춘3동,3742,3669,3704,3625,3504,3440,3305,3180,3083,2972,2823


In [57]:
child2['dong'] = child2['dong'].str[:2]
child2

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,옥련,3904,3715,3541,3332,3194,3067,2949,2765,2475,2315,2135
2,옥련,4145,3903,3694,3523,3387,3285,3172,2956,2772,2633,2420
3,선학,2348,2252,2154,1987,1868,1790,1722,1609,1501,1382,1332
4,연수,3819,3598,3413,3229,2987,2717,2486,2162,1856,1636,1517
5,연수,3558,3432,3288,3119,2980,2848,2768,2586,2397,2169,2042
6,연수,3145,3004,2908,2798,2521,2332,2191,2014,1777,1551,1428
7,청학,4221,3926,3697,3547,3287,3002,2842,2564,2273,2076,1921
8,동춘,2802,2676,2552,2366,2268,2162,2005,2404,3613,3620,3564
9,동춘,4160,3957,3796,3666,3469,3288,3183,2858,2570,2314,2189
10,동춘,3742,3669,3704,3625,3504,3440,3305,3180,3083,2972,2823


In [58]:
child3 = child2.groupby('dong').sum()

child3.reset_index(inplace=True)
child3

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘,10704,10302,10052,9657,9241,8890,8493,8442,9266,8906,8576
1,선학,2348,2252,2154,1987,1868,1790,1722,1609,1501,1382,1332
2,송도,12256,13787,15579,18642,21104,23449,25750,28780,32163,37592,38513
3,연수,10522,10034,9609,9146,8488,7897,7445,6762,6030,5356,4987
4,옥련,8049,7618,7235,6855,6581,6352,6121,5721,5247,4948,4555
5,청학,4221,3926,3697,3547,3287,3002,2842,2564,2273,2076,1921


In [59]:
dong_li = child3['dong']
dong_li2 = senior3['dong']
dong_li == dong_li2

0    True
1    True
2    True
3    True
4    True
5    True
Name: dong, dtype: bool

In [60]:
# 계산을 위해 object 삭제

child3.drop(['dong'],axis=1,inplace=True)
senior3.drop(['dong'],axis=1,inplace=True)


# 노령화지수 구하기
노령화지수 -> (15세 미만 / 65세 이상) * 100

In [61]:
# 노령화지수 -> (65세 이상 / 15세 미만) * 100
senior_child_rate = senior3 / child3 * 100
senior_child_rate['dong'] = dong_li+'동'
senior_child_rate

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,dong
0,38.705157,39.798097,42.409471,45.914880,48.988205,51.709786,56.646650,62.082445,65.130585,72.804851,81.576493,동춘동
1,79.216354,86.190053,94.103993,107.649723,119.914347,128.547486,143.263647,163.642014,186.608927,217.583213,240.240240,선학동
2,20.341057,20.998042,21.464792,20.904409,21.782600,22.239754,23.266019,24.127867,25.535553,26.883379,29.255057,송도동
3,53.126782,58.670520,64.200229,70.850645,78.051367,86.919083,97.447952,112.511091,133.515755,162.471994,182.414277,연수동
4,48.403528,52.966658,58.451970,63.267688,67.862027,72.921914,79.954256,89.022898,101.791500,115.137429,132.206367,옥련동
5,60.483298,70.860927,80.119015,86.552016,95.466991,109.393738,123.293455,144.617785,171.799384,202.023121,231.077564,청학동


In [62]:
# 2021년 노령화 지수만
senior_child_rate_2021 = senior_child_rate[['dong','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020', '2021']]
senior_child_rate_2021.columns=['dong','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
senior_child_rate_2021

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘동,38.705157,39.798097,42.409471,45.914880,48.988205,51.709786,56.646650,62.082445,65.130585,72.804851,81.576493
1,선학동,79.216354,86.190053,94.103993,107.649723,119.914347,128.547486,143.263647,163.642014,186.608927,217.583213,240.240240
2,송도동,20.341057,20.998042,21.464792,20.904409,21.782600,22.239754,23.266019,24.127867,25.535553,26.883379,29.255057
3,연수동,53.126782,58.670520,64.200229,70.850645,78.051367,86.919083,97.447952,112.511091,133.515755,162.471994,182.414277
4,옥련동,48.403528,52.966658,58.451970,63.267688,67.862027,72.921914,79.954256,89.022898,101.791500,115.137429,132.206367
5,청학동,60.483298,70.860927,80.119015,86.552016,95.466991,109.393738,123.293455,144.617785,171.799384,202.023121,231.077564


In [63]:
# 태블로 시각화용
senior3['dong'] = dong_li2
senior3

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,dong
0,4143,4100,4263,4434,4527,4597,4811,5241,6035,6484,6996,동춘
1,1860,1941,2027,2139,2240,2301,2467,2633,2801,3007,3200,선학
2,2493,2895,3344,3897,4597,5215,5991,6944,8213,10106,11267,송도
3,5590,5887,6169,6480,6625,6864,7255,7608,8051,8702,9097,연수
4,3896,4035,4229,4337,4466,4632,4894,5093,5341,5697,6022,옥련
5,2553,2782,2962,3070,3138,3284,3504,3708,3905,4194,4439,청학


In [64]:
# 재구성 및 정렬.

tableau_65세이상 = pd.melt(senior3,
               id_vars = 'dong',
               var_name = 'year',
               value_name = '65세이상')
tableau_65세이상= tableau_65세이상.sort_values(by=['dong','year'])
tableau_65세이상.reset_index(drop=True, inplace=True)
tableau_65세이상

,dong,year,65세이상
0,동춘,2011,4143
1,동춘,2012,4100
2,동춘,2013,4263
3,동춘,2014,4434
4,동춘,2015,4527
...,...,...,...
61,청학,2017,3504
62,청학,2018,3708
63,청학,2019,3905
64,청학,2020,4194


In [65]:
child3['dong'] = dong_li
child3

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,dong
0,10704,10302,10052,9657,9241,8890,8493,8442,9266,8906,8576,동춘
1,2348,2252,2154,1987,1868,1790,1722,1609,1501,1382,1332,선학
2,12256,13787,15579,18642,21104,23449,25750,28780,32163,37592,38513,송도
3,10522,10034,9609,9146,8488,7897,7445,6762,6030,5356,4987,연수
4,8049,7618,7235,6855,6581,6352,6121,5721,5247,4948,4555,옥련
5,4221,3926,3697,3547,3287,3002,2842,2564,2273,2076,1921,청학


In [66]:
# 재구성 및 정렬

tableau_15세미만 = pd.melt(child3,
               id_vars = 'dong',
               var_name = 'year',
               value_name = '15세미만')
tableau_15세미만= tableau_15세미만.sort_values(by=['dong','year'])
tableau_15세미만.reset_index(drop=True, inplace=True)
tableau_15세미만

,dong,year,15세미만
0,동춘,2011,10704
1,동춘,2012,10302
2,동춘,2013,10052
3,동춘,2014,9657
4,동춘,2015,9241
...,...,...,...
61,청학,2017,2842
62,청학,2018,2564
63,청학,2019,2273
64,청학,2020,2076


In [67]:
senior_child_rate_2021

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,동춘동,38.705157,39.798097,42.409471,45.914880,48.988205,51.709786,56.646650,62.082445,65.130585,72.804851,81.576493
1,선학동,79.216354,86.190053,94.103993,107.649723,119.914347,128.547486,143.263647,163.642014,186.608927,217.583213,240.240240
2,송도동,20.341057,20.998042,21.464792,20.904409,21.782600,22.239754,23.266019,24.127867,25.535553,26.883379,29.255057
3,연수동,53.126782,58.670520,64.200229,70.850645,78.051367,86.919083,97.447952,112.511091,133.515755,162.471994,182.414277
4,옥련동,48.403528,52.966658,58.451970,63.267688,67.862027,72.921914,79.954256,89.022898,101.791500,115.137429,132.206367
5,청학동,60.483298,70.860927,80.119015,86.552016,95.466991,109.393738,123.293455,144.617785,171.799384,202.023121,231.077564


In [68]:
# 노령화지수를 구하기 위해 2021년 데이터 가져오기
senior_child_final = senior_child_rate_2021

# 노령화지수 표준화 (쇠퇴진단지표 - 하한값) / (상한값 - 하한값) 
max2 = senior_child_final['2021'].max()
min2 = senior_child_final['2021'].min()
senior_child_final['std'] = (senior_child_final['2021'] - min2) / (max2 - min2)
senior_child_final

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,std
0,동춘동,38.705157,39.798097,42.409471,45.914880,48.988205,51.709786,56.646650,62.082445,65.130585,72.804851,81.576493,0.247986
1,선학동,79.216354,86.190053,94.103993,107.649723,119.914347,128.547486,143.263647,163.642014,186.608927,217.583213,240.240240,1.000000
2,송도동,20.341057,20.998042,21.464792,20.904409,21.782600,22.239754,23.266019,24.127867,25.535553,26.883379,29.255057,0.000000
3,연수동,53.126782,58.670520,64.200229,70.850645,78.051367,86.919083,97.447952,112.511091,133.515755,162.471994,182.414277,0.725924
4,옥련동,48.403528,52.966658,58.451970,63.267688,67.862027,72.921914,79.954256,89.022898,101.791500,115.137429,132.206367,0.487955
5,청학동,60.483298,70.860927,80.119015,86.552016,95.466991,109.393738,123.293455,144.617785,171.799384,202.023121,231.077564,0.956572


In [69]:
tableau_senior_child_final =pd.melt(senior_child_final.drop(['std'],axis=1),
               id_vars = 'dong',
               var_name = 'year',
               value_name = 'senior_rate')
tableau_senior_child_final = tableau_senior_child_final.sort_values(by=['dong','year'])

tableau_senior_child_final.reset_index(drop=True, inplace=True)
tableau_senior_child_final

,dong,year,senior_rate
0,동춘동,2011,38.705157
1,동춘동,2012,39.798097
2,동춘동,2013,42.409471
3,동춘동,2014,45.914880
4,동춘동,2015,48.988205
...,...,...,...
61,청학동,2017,123.293455
62,청학동,2018,144.617785
63,청학동,2019,171.799384
64,청학동,2020,202.023121


# 복합쇠퇴지수 용 데이터 내보내기

In [70]:
total_pop_final

,dong,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,9년 인구증감율,인구상대감소지수_std
0,동춘동,59575.0,58857.0,58510.0,57096.0,56343.0,57806.0,61872.0,64428.0,62780.0,61722.0,3.603861,0.875803
1,선학동,20812.0,20522.0,20126.0,19679.0,19482.0,18798.0,18323.0,17689.0,17296.0,17012.0,-18.258697,0.978473
2,송도동,65121.0,78431.0,86297.0,103650.0,116124.0,126949.0,144395.0,163762.0,188166.0,188914.0,190.096897,0.000000
3,연수동,72264.0,71867.0,70412.0,68606.0,66907.0,64201.0,61765.0,59648.0,57500.0,55757.0,-22.842633,1.000000
4,옥련동,46946.0,46322.0,45744.0,45422.0,45074.0,44148.0,42347.0,41449.0,40194.0,39269.0,-16.352831,0.969523
5,청학동,30868.0,30755.0,30227.0,29650.0,28919.0,27716.0,26516.0,25590.0,24891.0,24629.0,-20.211870,0.987645


In [71]:
total_pop_final.to_csv('최종_변환_인구사회_인구증감률_2013-2022.csv')

In [72]:
basic_final

,emd_nm,2021년 천명당 기초생활수급자,2021년 천명당 기초생활수급자_std
0,동춘동,15.036636,0.126928
1,선학동,94.010176,1.000000
2,송도동,3.555371,0.000000
3,연수동,86.000000,0.911446
4,옥련동,40.354282,0.406821
5,청학동,79.948576,0.844546


In [73]:
basic_final.to_csv('최종_변환_인구사회_천명당기초생활수급자수.csv')

In [74]:
senior_child_final

,dong,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,std
0,동춘동,38.705157,39.798097,42.409471,45.914880,48.988205,51.709786,56.646650,62.082445,65.130585,72.804851,81.576493,0.247986
1,선학동,79.216354,86.190053,94.103993,107.649723,119.914347,128.547486,143.263647,163.642014,186.608927,217.583213,240.240240,1.000000
2,송도동,20.341057,20.998042,21.464792,20.904409,21.782600,22.239754,23.266019,24.127867,25.535553,26.883379,29.255057,0.000000
3,연수동,53.126782,58.670520,64.200229,70.850645,78.051367,86.919083,97.447952,112.511091,133.515755,162.471994,182.414277,0.725924
4,옥련동,48.403528,52.966658,58.451970,63.267688,67.862027,72.921914,79.954256,89.022898,101.791500,115.137429,132.206367,0.487955
5,청학동,60.483298,70.860927,80.119015,86.552016,95.466991,109.393738,123.293455,144.617785,171.799384,202.023121,231.077564,0.956572


In [75]:
senior_child_final.to_csv('최종_변환_인구사회_노령화지수_2021.csv')
# senior_child_final.to_csv('태블로_인구사회_노령화지수_노령화지수 2011-2021.csv')

# 테블로 시각화용 내보내기

In [76]:
# 인구증감률
tableau_총인구수2 = tableau_총인구수[['year','emd_nm','총인구수']]
tableau_총인구수2.columns=['year','dong','pop']
tableau_총인구수2

,year,dong,pop
0,2013,동춘동,59575.0
1,2014,동춘동,58857.0
2,2015,동춘동,58510.0
3,2016,동춘동,57096.0
4,2017,동춘동,56343.0
5,2018,동춘동,57806.0
6,2019,동춘동,61872.0
7,2020,동춘동,64428.0
8,2021,동춘동,62780.0
9,2022,동춘동,61722.0


In [77]:
tableau_총인구수2=tableau_총인구수2.sort_values(by=['dong','year'])
tableau_총인구수2.reset_index(drop=True, inplace=True)
tableau_총인구수2

,year,dong,pop
0,2013,동춘동,59575.0
1,2014,동춘동,58857.0
2,2015,동춘동,58510.0
3,2016,동춘동,57096.0
4,2017,동춘동,56343.0
5,2018,동춘동,57806.0
6,2019,동춘동,61872.0
7,2020,동춘동,64428.0
8,2021,동춘동,62780.0
9,2022,동춘동,61722.0


In [78]:
tableau_총인구수2.to_csv('총인구수_2013-2022.csv')

In [79]:
# 노령화지수
tableau_senior_child_final

,dong,year,senior_rate
0,동춘동,2011,38.705157
1,동춘동,2012,39.798097
2,동춘동,2013,42.409471
3,동춘동,2014,45.914880
4,동춘동,2015,48.988205
...,...,...,...
61,청학동,2017,123.293455
62,청학동,2018,144.617785
63,청학동,2019,171.799384
64,청학동,2020,202.023121


In [80]:
tableau_senior_child_final.to_csv('노령화지수_2011-2021')

In [81]:
tableau_기초생활수급자수.columns=['dong','year','천명당기초생활수급자']
tableau_기초생활수급자수

,dong,year,천명당기초생활수급자
0,동춘동,2012,265.0
1,동춘동,2013,296.0
2,동춘동,2014,298.0
3,동춘동,2015,350.0
4,동춘동,2016,485.0
...,...,...,...
61,청학동,2018,1505.0
62,청학동,2019,1627.0
63,청학동,2020,1732.0
64,청학동,2021,1990.0


In [82]:
tableau_기초생활수급자수.to_csv('천명당기초생활수급자수_change.csv')